In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX

### Lấy biến y từ dữ liệu

In [3]:
# y = pd.read_excel('Optimal_MB_dataset.xlsx', sheet_name='Data')[['DIVIDENT']][4:].to_numpy().flatten()
y = pd.read_excel('Simulated_TS_data.xlsx')[['DIVIDENT_GROWTH']][4:].to_numpy().flatten()

### Lấy các biến x và tạo các biến transformed

In [17]:
list_var = ['GDP', 'PDI', 'PCE', 'PROFIT']

# X0 = pd.read_excel('Optimal_MB_dataset.xlsx', sheet_name='Data')[list_var]
X0 = pd.read_excel('Simulated_TS_data.xlsx')[list_var]

for var in list_var:
    X0[var+'_LAG1'] = X0[var].shift(1)
    X0[var+'_LAG2'] = X0[var].shift(2)
    X0[var+'_GROWTH'] = (X0[var]-X0[var].shift(1))/X0[var]
    X0[var+'_GROWTH1'] = (X0[var]-X0[var].shift(4))/X0[var]
    X0[var+'_LN'] = np.log(X0[var])
    X0[var+'_LOG10'] = np.log10(X0[var])

# lấy dữ liệu đầu vào mô hình từ dòng thứ 4,
# vì biến _GROWTH1 là tăng trưởng so với quý cùng kỳ năm trước nên dữ liệu sẽ mất năm đầu tiên
X = X0.iloc[4:,]

In [18]:
X

,GDP,PDI,PCE,PROFIT,GDP_LAG1,GDP_LAG2,GDP_GROWTH,GDP_GROWTH1,GDP_LN,GDP_LOG10,...,PCE_GROWTH,PCE_GROWTH1,PCE_LN,PCE_LOG10,PROFIT_LAG1,PROFIT_LAG2,PROFIT_GROWTH,PROFIT_GROWTH1,PROFIT_LN,PROFIT_LOG10
4,1.224976,1.095450,1.007006,0.999399,1.147886,1.197667,0.062932,0.169895,0.202921,0.088128,...,-0.221818,-0.011862,0.006982,0.003032,1.117113,1.200759,-0.117785,-0.089601,-0.000601,-0.000261
5,1.363940,1.174860,0.952497,1.050569,1.224976,1.147886,0.101884,0.182706,0.310378,0.134795,...,-0.057228,-0.157414,-0.048669,-0.021137,0.999399,1.117113,0.048707,-0.067692,0.049332,0.021425
6,1.446646,1.247493,1.052122,0.945158,1.363940,1.224976,0.057171,0.172108,0.369248,0.160362,...,0.094690,-0.109136,0.050809,0.022066,1.050569,0.999399,-0.111527,-0.270432,-0.056403,-0.024495
7,1.272123,1.269539,1.104924,1.058557,1.446646,1.363940,-0.137191,0.097661,0.240687,0.104529,...,0.047789,-0.113541,0.099777,0.043333,0.945158,1.050569,0.107125,-0.055318,0.056906,0.024714
8,1.219608,1.403456,1.034085,1.059426,1.272123,1.446646,-0.043059,-0.004401,0.198530,0.086220,...,-0.068504,0.026187,0.033517,0.014556,1.058557,0.945158,0.000820,0.056659,0.057727,0.025070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,6.215351,13.828305,0.121361,0.009180,6.079846,6.601911,0.021802,0.070673,1.827022,0.793466,...,-0.112812,0.017157,-2.108985,-0.915920,0.009347,0.008427,-0.018236,0.030761,-4.690755,-2.037169
996,5.528375,12.770980,0.116694,0.008274,6.215351,6.079846,-0.124264,-0.229317,1.709894,0.742598,...,-0.039999,-0.019347,-2.148204,-0.932953,0.009180,0.009347,-0.109407,-0.051488,-4.794581,-2.082260
997,5.169229,13.787088,0.110231,0.008773,5.528375,6.215351,-0.069478,-0.277156,1.642724,0.713426,...,-0.058628,-0.193228,-2.205178,-0.957697,0.008274,0.009180,0.056833,0.039492,-4.736069,-2.056849
998,4.484431,15.960048,0.108044,0.010465,5.169229,5.528375,-0.152706,-0.355768,1.500612,0.651707,...,-0.020239,-0.249971,-2.225216,-0.966399,0.008773,0.008274,0.161657,0.106799,-4.559741,-1.980270


### Kiểm tra VIF của từng biến, loại các biến có VIF > 5

In [ ]:
# vif = pd.DataFrame()
# vif['Variable'] = X.columns
# vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(vif.shape[0])]
# max_vif = max(vif['VIF'])
# while max_vif > 5:
#     var = vif[vif['VIF'] == max_vif]['Variable']
#     X = X.drop(columns=var)
#     vif = vif[vif['VIF'] != max_vif]
#     vif = vif.reset_index(drop=True)
#     vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(vif.shape[0])]
#     max_vif = max(vif['VIF'])
# vif

c:\Users\VR685BL\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


,Variable,VIF
0,GDP,1.455880
1,PROFIT,2.220429
2,GDP_GROWTH,1.352126
3,GDP_GROWTH1,1.388790
4,PDI_LAG2,1.542448
5,PDI_GROWTH,1.324826
6,PDI_GROWTH1,1.348973
7,PCE_LAG2,2.598248
8,PCE_GROWTH,1.355004
9,PCE_GROWTH1,1.358435


### Kruskal - Wallis test

In [19]:
from scipy.stats import kruskal

kw = kruskal(*[X[var] for var in X.columns])

print(str(kw) + '\n\n=> The time series '.format(kw[1]), end='')
if kw[1] < 0.05:
    print('do not ', end='')
print('come from the same distribution.')

KruskalResult(statistic=19642.154348316366, pvalue=0.0)

=> The time series do not come from the same distribution.


In [20]:
def mape(lin_reg, x, y):
    return sum(np.abs((y - lin_reg.predict())/y))/len(y)

### Standardizing the dataset

In [21]:
for col in X.columns:
    X[col] = (X[col] - X[col].mean()) / X[col].std()

y = (y - y.mean()) / y.std()

C:\Users\VR685BL\AppData\Local\Temp\ipykernel_28620\2180267628.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = (X[col] - X[col].mean()) / X[col].std()


### Chia tập dữ liệu dữ liệu train và test

In [22]:
split = int(X.shape[0] * 0.78)

X_train = X.iloc[:split,:]
X_test = X.iloc[split:,:]

y_train = y[:split]
y_test = y[split:]

### Chạy lại hồi quy linear và lưu R-squared cho từng biến một, sau đó sắp xếp các biến theo R2 giảm dần

In [23]:
r2 = pd.DataFrame(columns=['Var', 'p-value', 'R2', 'MAPE'])
for var in X_train.columns:
    # chạy linear regression từng biến một
    lin_reg = sm.OLS(y_train, sm.add_constant(X_train[var])).fit()
    # liệt kê tên biến, R2 và p-value tương ứng vào bảng r2
    r2.loc[-1] = [var, lin_reg.summary2().tables[1]['P>|t|'].iloc[1], lin_reg.rsquared, mape(lin_reg, sm.add_constant(X_train[var]), y_train)]
    r2.index += 1
# sắp xếp lại các biến theo R2 từ cao đến thấp
r2 = r2.sort_values(by=['R2'], ascending=False)
r2 = r2.reset_index(drop=True)
r2

,Var,p-value,R2,MAPE
0,PROFIT_LOG10,9.906849e-53,0.260636,1.620326
1,PROFIT_LN,9.906849e-53,0.260636,1.620326
2,PROFIT_LAG2,4.710986e-47,0.235335,1.723521
3,PROFIT_LAG1,9.800041e-47,0.233892,1.723686
4,PROFIT,2.129162e-46,0.232361,1.722068
5,PDI_LOG10,5.235456e-30,0.154170,1.540231
6,PDI_LN,5.235456e-30,0.154170,1.540231
7,PCE,4.545493e-29,0.149467,1.621251
8,PCE_LAG1,8.725178e-29,0.148043,1.591033
9,PCE_LAG2,1.891411e-28,0.146350,1.598088


In [ ]:
# Define the family of each variable
family = {}
for var in r2['Var']:
    for fam in list_var:
        if var.startswith(fam):
            family[var] = fam

In [ ]:
def stepwise_selection(X, y, initial_list=[], threshold_in=0.05, threshold_out=0.05, verbose=True):
    included = list(initial_list)
    family_included = set()
    
    while True:
        changed = False
        # loại những biến cùng họ đã có trong final list (included)
        excluded = list(set(X.columns) - set(included))
        excluded_sorted = r2[r2['Var'].isin(excluded)].sort_values(by='R2', ascending=False)['Var']
        new_pval = pd.Series(index=excluded_sorted)
        # thử đẩy từng biến vào mô hình theo thứ tự R2 giảm dần
        for new_column in excluded_sorted:
            if family[new_column] in family_included:
                continue
            print('-> Trying combination ' + str(included + [new_column]))
            auto_arima_model = auto_arima(y_train, X=X[included + [new_column]], 
                                        start_p=0, d=0, start_q=0, 
                                        max_p=7, max_d=2, max_q=7, 
                                        start_P=0, D=0, start_Q=0, 
                                        max_P=7, max_D=2, max_Q=7, 
                                        m=7, seasonal=True, information_criterion='aic', 
                                        stepwise=True, error_action='warn',trace=False, 
                                        random_state=64, n_fits=50, with_intercept='auto') 

            # fit mô hình ARIMAX với bậc của auto_arima
            arimax_model = SARIMAX(y_train, exog=X[included + [new_column]], order=auto_arima_model.order, seasonal_order = auto_arima_model.seasonal_order).fit(disp=0)
            new_pval[new_column] = arimax_model.pvalues[new_column]
            # print(arimax_model.pvalues)
        
        # lấy p-value nhỏ nhất, nếu nhỏ hơn threshold 0.05 sẽ cho biến đó vào included list
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            family_included.add(family[best_feature])
            changed = True
            if verbose:
                print(f'Add  {best_feature:30} with p-value {best_pval:.6}')

        # backward step
        auto_arima_model = auto_arima(y_train, X=X[included], 
                                    start_p=0, d=0, start_q=0, 
                                    max_p=7, max_d=2, max_q=7, 
                                    start_P=0, D=0, start_Q=0, 
                                    max_P=7, max_D=2, max_Q=7, 
                                    m=7, seasonal=True, information_criterion='aic', 
                                    stepwise=True, error_action='warn',trace=False, 
                                    random_state=64, n_fits=50, with_intercept='auto')
        arimax_model = SARIMAX(y_train, exog=X[included], order=auto_arima_model.order, seasonal_order = auto_arima_model.seasonal_order).fit(disp=False)
        pvalues = arimax_model.pvalues

        # sau khi lấy biến mới vào final list, check lại xem p-value các biến trong final list có tăng không, nếu tăng sẽ bỏ
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            family_included.remove(family[worst_feature])
            if verbose:
                print(f'Drop {worst_feature:30} with p-value {worst_pval:.6}')
        if not changed:
            break

    return included

In [35]:
print(stepwise_selection(X_train, y_train))

Trying combination ['PROFIT_LOG10']
PROFIT_LOG10     7.720571e-01
ar.L1            0.000000e+00
ar.L2            3.102174e-05
sigma2          4.157637e-223
dtype: float64
Trying combination ['PROFIT_LN']
PROFIT_LN     7.720571e-01
ar.L1         0.000000e+00
ar.L2         3.102175e-05
sigma2       4.157638e-223
dtype: float64
Trying combination ['PROFIT_LAG2']
PROFIT_LAG2     9.447473e-01
ar.L1           0.000000e+00
ar.L2           3.256473e-05
sigma2         5.666673e-223
dtype: float64
Trying combination ['PROFIT_LAG1']
PROFIT_LAG1     9.400215e-01
ar.L1           0.000000e+00
ar.L2           3.326858e-05
sigma2         3.967986e-222
dtype: float64
Trying combination ['PROFIT']
PROFIT     9.485814e-01
ar.L1      0.000000e+00
ar.L2      3.337167e-05
sigma2    5.689207e-223
dtype: float64
Trying combination ['PDI_LOG10']
PDI_LOG10     3.865063e-01
ar.L1         0.000000e+00
ar.L2         3.011334e-05
sigma2       3.569403e-216
dtype: float64
Trying combination ['PDI_LN']
PDI_LN     3.8